In [4]:
import os

print(f'current working directory: {os.getcwd()}')
os.chdir('../')
print(f'updated working directory: {os.getcwd()}')


current working directory: d:\MLOPs-Tutorials\DS-Project-end2end\research
updated working directory: d:\MLOPs-Tutorials\DS-Project-end2end


In [5]:
from dataclasses import dataclass
from pathlib import Path

In [16]:
@dataclass
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    #these params will be initialised in params.yaml
    alpha:float
    l1_ratio:float
    target_column:str

In [17]:
from src.dataScience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.dataScience.utils.common import read_yaml, create_directory

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directory([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer #config dict se hum konsi key use krenge!!
        params=self.params.ElasticNet
        schema=self.schema.target_column
        create_directory([config.root_dir])
        
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path, 
            test_data_path=config.test_data_path, 
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column = schema.name

        )
        return model_trainer_config

In [37]:
import os
from src.dataScience import logger
from sklearn.linear_model import ElasticNet
import joblib
import pandas as pd

In [34]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        train_x=train_data.drop([self.config.target_column], axis=1)
        test_x=test_data.drop([self.config.target_column], axis=1)
        
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]   
        
        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [35]:
schema_path = Path('schema.yaml')
print(schema_path.resolve())  # absolute path
schema = read_yaml(schema_path)
print(schema.keys())
print(schema)

D:\MLOPs-Tutorials\DS-Project-end2end\schema.yaml
[2025-08-10 01:35:58,726: INFO: common: yaml file: schema.yaml loaded succesfully]
dict_keys(['COLUMNS', 'target_column'])
{'COLUMNS': {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}, 'target_column': {'name': 'quality'}}


In [39]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-08-10 01:37:08,381: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-08-10 01:37:08,382: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-08-10 01:37:08,384: INFO: common: yaml file: schema.yaml loaded succesfully]
[2025-08-10 01:37:08,386: INFO: common: created directory at: artifacts]
[2025-08-10 01:37:08,388: INFO: common: created directory at: artifacts/model_trainer]
